In [1]:
#
# Import Libraries
#

import numpy as np
import os
import pandas as pd
import sys

from plotly.subplots import make_subplots
import plotly.graph_objects as go

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

from myLog import myLog
from PlotEDA import PlotEDA
from PlotMV import PlotMV
from PCA_UMAP import PCA_UMAP

In [2]:
#
# Constants
#

MVF_thr = 0.2
MVO_thr = 0.1

In [3]:
#
# Set constants
#

workingPath = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA"
c18p_path = os.path.join(workingPath, 'OriginalFiles', 'RBR_V1_C18_POS_LOESS_featureData.xlsx')
c18n_path = os.path.join(workingPath, 'OriginalFiles', 'RBR_V1_C18_NEG_LOESS_featureData.xlsx')
hpn_path = os.path.join(workingPath, 'OriginalFiles', 'RBR_V1_HILIC_POS-NEG_LOESS_featureData.xlsx')
code2sn_path = os.path.join(workingPath, 'OriginalFiles', 'V1_Metab.xlsx')

fileSummary = os.path.join(workingPath, 'WorkingFiles', 'Plots', 'SummaryPlots.html')
fileCohort = os.path.join(workingPath, 'WorkingFiles', 'Plots', 'CohortPlots.html')
fileGroup = os.path.join(workingPath, 'WorkingFiles', 'Plots', 'GroupPlots.html')
filePCA = os.path.join(workingPath, 'WorkingFiles', 'Plots', 'PCAPlots.html')
if os.path.exists(fileSummary): os.remove(fileSummary)
if os.path.exists(fileCohort): os.remove(fileCohort)
if os.path.exists(fileGroup): os.remove(fileGroup)
if os.path.exists(filePCA): os.remove(filePCA)

In [4]:
#
# Set logging
#

logw = myLog(os.path.join(workingPath,'WorkingFiles', 'info.log'))
logw('Start Session')

Start Session


In [5]:
#
# Read tables
#

c18p = pd.read_excel(c18p_path)
c18n = pd.read_excel(c18n_path)
hpn = pd.read_excel(hpn_path)
code2sn = pd.read_excel(code2sn_path)

In [6]:
#
# Create f2info.tsv
#

# Add fid
c18p['fid'] = [f"C18P{i+1}" for i in range(c18p.shape[0])]
c18n['fid'] = [f"C18N{i+1}" for i in range(c18n.shape[0])]

In [7]:
from itertools import count
pcount = count(1,1)
ncount = count(1,1)
hpn['fid'] = [f"HILP{next(pcount)}" if i[1]=='P' else f"HILN{next(ncount)}" for i in hpn['Name']]

In [8]:
f2i_c18p = c18p.loc[:, ['fid', 'Name', 'Apex m/z', 'RT [min]']]
f2i_c18p['column'] = 'C18'
f2i_c18p['mode'] = 'POS'
f2i_c18n = c18n.loc[:, ['fid', 'Name', 'Apex m/z', 'RT [min]']]
f2i_c18n['column'] = 'C18'
f2i_c18n['mode'] = 'NEG'
f2i_hpn = hpn.loc[:, ['fid', 'Name', 'Apex m/z', 'RT [min]']]
f2i_hpn['column'] = 'HILIC'
f2i_hpn['mode'] = ["POS" if i[1]=='P' else "NEG" for i in hpn['Name']]

f2i = pd.concat([
    f2i_c18p,
    f2i_c18n,
    f2i_hpn
])

f2i.to_csv(os.path.join(workingPath, 'WorkingFiles', 'f2info.tsv'), sep='\t', index=False)

In [9]:
# Maintain fid only

[i.drop(['Name', 'Apex m/z', 'RT [min]'], inplace=True, axis=1) for i in [c18p, c18n, hpn]]

[None, None, None]

In [10]:
#
# Add Seqn to HILIC file
#

hpnT = hpn.set_index('fid').T

code2sn['code'] = [i.split('_')[1] for i in code2sn['Name']]
code2sn = code2sn.set_index('code')

hpnT = hpnT.set_index(code2sn.loc[[i.split('_')[1] for i in hpnT.index]]['Seqn'])

In [11]:
# Create Xm.tsv

c18pT = c18p.set_index('fid').T
c18nT = c18n.set_index('fid').T

xm = code2sn.loc[:, ['Seqn']].set_index('Seqn')

for i in [c18pT, c18nT, hpnT]:
    xm = xm.join(i)

In [12]:
xm.to_csv(os.path.join(workingPath, 'WorkingFiles', 'Xm.tsv'), sep='\t')

In [13]:
#
# filter by missing values, center&scale, impute missing values, check batch effect
#

In [14]:
mdata = pd.read_csv(r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\PESA\WorkingFiles\main_metadata.tsv', sep='\t')

In [15]:
plotMV = PlotMV(xm, mdata, file=fileSummary)
plotMV.plotSummary()

In [16]:
logw('')
logw(f"Total number of observations: {xm.shape[0]}")
logw(f"Total number of features: {xm.shape[1]}")
logw(f"Total number of features with <{MVF_thr*100}% of missing values(<{int(xm.shape[0]*MVF_thr)} of obs.): {((xm.isna().sum()/xm.shape[0])<=MVF_thr).sum()}")


Total number of observations: 444
Total number of features: 2611
Total number of features with <20.0% of missing values(<88 of obs.): 2611


In [17]:
print(f2i.loc[:, ['column', 'mode']].groupby(['column']).size())
print()
print(f2i.loc[:, ['column', 'mode']].groupby(['column', 'mode']).size())

column
C18      1557
HILIC    1054
dtype: int64

column  mode
C18     NEG      232
        POS     1325
HILIC   NEG      591
        POS      463
dtype: int64


In [18]:
#
# Filter by missing values (although no feature will be removed)
#

xmf = xm.loc[:, xm.isna().sum()/xm.shape[0] <= MVF_thr]

In [19]:
#
# Summary plots
#

plotEDA = PlotEDA(xmf, mdata, file=fileSummary)
plotEDA.plotSummary(
    r11=(0.75, 1.25), r12=(0,0.2), 
    r3=(-2,2), ry3=(0,1),
    vl3=[0],
    binsize=0.01,
    titleLabel=f''
)

plotEDA = PlotEDA(xmf, mdata, file=fileCohort)
plotEDA.plotByGroup('Cohort',vl1=[0],vl2=[1], r1=(0.9,1.1), r2=(-6,6), binsize=0.0005, plotN=True)
plotEDA = PlotEDA(xmf, mdata, file=fileGroup)
plotEDA.plotByGroup('Group',vl1=[0],vl2=[1], r1=(0.9,1.1), r2=(-6,6), binsize=0.0005, plotN=True)

In [20]:
#
# Filter observations by Missing values
#

plotMV = PlotMV(xmf, mdata, file=fileSummary)
plotMV.plotSummaryObs()

# Filter Observations by missing values

xmf = xmf[xmf.isna().sum(axis=1)/xmf.shape[1]<MVO_thr]

logw(f'Total number of observations with <{MVO_thr*100}% of missing values: {xmf.shape[0]} / {xm.shape[0]}')

Total number of observations with <10.0% of missing values: 383 / 444


In [21]:
#
# Standardize
#

from sklearn.preprocessing import StandardScaler, RobustScaler

xmfn = pd.DataFrame(
    StandardScaler().fit_transform(xmf),
    columns=xmf.columns, index=xmf.index
)

In [22]:
#
# Imputation of missing values using KNN
#

from sklearn.impute import KNNImputer

xmfnv = pd.DataFrame(
    KNNImputer(n_neighbors=3).fit_transform(xmfn),
    columns=xmfn.columns,
    index=xmfn.index
)

In [23]:
print(f"Imputed missing values: {xmfn.isna().sum().sum()}/{xmfn.shape[0]*xmfn.shape[1]} ({round(xmfn.isna().sum().sum()/(xmfn.shape[0]*xmfn.shape[1])*100, 2)}%)")

Imputed missing values: 0/1000013 (0.0%)


In [24]:
#
# Correct Batch Effect
# Comparamos batch effect correction con los dos tipos de escalado

# from combat import combat
# from scipy.stats import kruskal, median_test

# xmfnvb = combat(
#     data=xmfnv.T,
#     batch=mdata.set_index('Seqn').loc[xmfnv.index, 'Cohort']
# ).T

from myComBat import myComBat

catVars = ['Group', 'Smoke']
conVars = ['Calcium_Score', 'HDL', 'LDL', 'Total_Cholesterol','Ox-LDL','Lipoprotein(a)','CRP', 'Plaque_thickness']
xmfnvb = myComBat(xmfnv, mdata, 'Cohort', catVars, conVars, Rpath=os.path.join(workingPath, 'WorkingFiles', 'myRData'))

Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.
Loading required package: genefilter
Loading required package: BiocParallel
Warning message:
package 'BiocParallel' was built under R version 4.2.1 
Found4batches
Adjusting for12covariate(s) or covariate level(s)
Standardizing Data across genes
Fitting L/S model and finding priors
Finding nonparametric adjustments
Adjusting the Data




In [25]:
plotEDA = PlotEDA(xmfnvb, mdata, file=fileCohort)
plotEDA.plotByGroup('Cohort',vl1=[0],vl2=[1], r1=(-6,6),binsize=0.0005, plotN=False)
plotEDA = PlotEDA(xmfnvb, mdata, file=fileGroup)
plotEDA.plotByGroup('Group',vl1=[0],vl2=[1], r1=(-6,6),binsize=0.0005, plotN=False)

In [26]:
# plotEDA = PlotEDA(xmfnvb, mdata)#, file=fileCohort)
# plotEDA.plotByGroup('Cohort',vl1=[0],vl2=[1], r1=(-6,6), r2=(-6,6), binsize=0.0005, plotN=False)

In [27]:
plotEDA._kruskal(xmfnvb, 'Cohort')

KruskalResult(statistic=57.44098227377981, pvalue=2.0690579817047877e-12)

In [28]:
#
# Generate 
#

xmfnvb.to_csv(os.path.join(workingPath, 'WorkingFiles', 'Xm_norm.tsv'), sep='\t')

In [29]:
#
# Dimensionality Reduction
#

pcaumap = PCA_UMAP(xmfnv, mdata, file=filePCA)
pcaumap.plotReduction('Cohort', pcacomp=[0,1])
pcaumap.plotReduction('Group', pcacomp=[0,1])

pcaumap = PCA_UMAP(xmfnvb, mdata, file=filePCA)
pcaumap.plotReduction('Cohort', pcacomp=[0,1], titleLabel='- Batch Corrected')
pcaumap.plotReduction('Group', pcacomp=[0,1], titleLabel='- Batch Corrected')